In [1]:
import numpy as np

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from nltk.corpus import stopwords
Stw = set(stopwords.words('english')) #stopwords in english


In [4]:
import string

#reading a .txt file and returning a dictianary of words 
def Word_freq(path_art):

# Open the file in read mode

    with open(path_art, "r",encoding='Windows-1252') as text:

        # Create an empty dictionary
        d = {}

        # Loop through each line of the file
        l=0
        for line in text:
            l+=1
            if l<15:
                continue

            #print(type(line))
            # Remove the leading spaces and newline character
            line = line.strip()

            # Convert the characters in line to
            # lowercase to avoid case mismatch
            line = line.lower()

            # Remove the punctuation marks from the line
            line = line.translate(line.maketrans("", "", string.punctuation))

            # Split the line into words
            words = line.split(" ")

            # Iterate over each word in line
            
            for word in words:
                #vomit the word if the word is stopword or numeric
                if word in Stw or word.isnumeric() or word=='' or word==' ':
                    continue
                # Check if the word is already in dictionary 
                if word in d:
                    # Increment count of word by 1
                    d[word] = d[word] + 1
                else:
                    # Add the word to dictionary with count 1
                    d[word] = 1

        return d
        

In [5]:
import os
path_root = r"20_newsgroups"
List_sub = os.listdir(path_root)

dict_freq = {}
total_files =0
#traversing through files of news articles
for sub in List_sub:
    path_sub = path_root+"\\"+sub
    List_art = os.listdir(path_sub)
    for art in List_art:
        path_art = path_sub + "\\" + art
        total_files+=1
        d = Word_freq(path_art)
        for key in d:
            if key not in dict_freq:
                dict_freq[key]=0
            dict_freq[key]+=d[key]

            
        


In [4]:
#making a list of features(words) whose frequency is greater that 4
Feature =[] 
for key in dict_freq:
    if dict_freq[key] > 4:
        Feature.append([key,dict_freq[key]])


In [5]:
Feature.sort( key = lambda x: x[1], reverse=1)

In [6]:
Feature_indx ={}
Feature_ls=[]
for i in range(len(Feature)):
    Feature_indx[Feature[i][0]]=i
    Feature_ls.append(Feature[i][0])

In [9]:
#preparing a clean dataset
def process_text(path_root):
    List_sub = os.listdir(path_root)
    data_tr_ls=[]
    tabular_result=[]
    for sub in List_sub:
        path_sub = path_root+"\\"+sub
        List_art = os.listdir(path_sub)
        for art in List_art:
            path_art = path_sub + "\\" + art
            d = Word_freq(path_art)
            art_ls=[0]*len(Feature_ls)
            for key in d:
                if key in Feature_indx:
                    ind = Feature_indx[key]
                    art_ls[ind]+=1
            data_tr_ls.append(art_ls)
            tabular_result.append(sub)
    return data_tr_ls, tabular_result

In [10]:
path_root = r"20_newsgroups"
data_tr_ls, data_result_ls =  process_text(path_root)


In [13]:
data_target = np.array(data_tr_ls)

In [14]:
import gc
del data_tr_ls
gc.collect()

29

In [15]:
result_dict ={}
result_num_to_str_map=[]
for i in range(len(data_result_ls)):
    if data_result_ls[i] not in result_dict:
        result_num_to_str_map.append(data_result_ls[i])
        result_dict[data_result_ls[i]] = len(result_dict)
    data_result_ls[i]= result_dict[data_result_ls[i]]
        
    


In [16]:
data_result = np.array(data_result_ls)

In [17]:
del data_result_ls
del Feature
del Feature_ls
del dict_freq
del result_dict
gc.collect()

0

Now 'data_target' & 'data_result' is final dataset

In [18]:
from sklearn import model_selection
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(data_target, data_result,test_size=0.1)

In [19]:
del data_result
del data_target
gc.collect()

0

In [20]:
import math

# Multinomial Naive Bayes made from scratch

In [21]:
class Mult_NB:


    def fit(self, X_train , Y_train):
        total_classes = len(set(Y_train))
        self.count_arr = np.ones((total_classes, X_train.shape[1]))
        self.count_in_1class = np.full( (total_classes,) ,X_train.shape[1])
        self.class_freq  = np.zeros((total_classes,))
        self.train_size= Y_train.shape[0]
        for i in range(Y_train.shape[0]):
            for j in range(X_train.shape[1]):
                self.count_arr[Y_train[i]][j]+=X_train[i][j]
                self.count_in_1class[Y_train[i]] += X_train[i][j]
            self.class_freq[Y_train[i]]+=1
        


    def predict(self, X_test):
        Y_pred = np.zeros(X_test.shape[0])
        for i in range(Y_pred.shape[0]):
            max_prob=-1
            predic=-1
            first_itr= True
            for possible_pred in range(self.class_freq.shape[0]):
                prob_local=0
                for j in range(X_test.shape[1]):
                    if X_test[i][j]>0:
                        prob_local += math.log(self.count_arr[possible_pred][j] / self.count_in_1class[possible_pred])
                    else:
                        prob_local += math.log(1 - ( self.count_arr[possible_pred][j] / self.count_in_1class[possible_pred] )  )
                prob_local+= math.log(self.class_freq[possible_pred] / self.train_size)
                if first_itr or prob_local > max_prob:
                    max_prob= prob_local
                    predic = possible_pred
                    first_itr = False
            Y_pred[i]=predic
        return Y_pred
        



In [22]:
clf_scratch = Mult_NB()
clf_scratch.fit(X_train, Y_train)

In [24]:
Y_pred_scratch = clf_scratch.predict(X_test)

In [29]:
from sklearn.metrics import classification_report
print("report of model made from scratch:")
print(classification_report(Y_test, Y_pred_scratch, target_names=result_num_to_str_map))

report of model made from scratch:
                          precision    recall  f1-score   support

             alt.atheism       0.65      0.74      0.69        87
           comp.graphics       0.73      0.67      0.70       111
 comp.os.ms-windows.misc       0.79      0.80      0.79       105
comp.sys.ibm.pc.hardware       0.63      0.65      0.64        98
   comp.sys.mac.hardware       0.72      0.71      0.72        89
          comp.windows.x       0.74      0.77      0.76        96
            misc.forsale       0.78      0.73      0.75       115
               rec.autos       0.86      0.87      0.87        86
         rec.motorcycles       0.95      0.91      0.93       102
      rec.sport.baseball       0.95      0.85      0.89       110
        rec.sport.hockey       0.88      0.93      0.90        84
               sci.crypt       0.91      0.90      0.91        93
         sci.electronics       0.81      0.69      0.75        94
                 sci.med       0.92     

# Multinomial Naive Bayes by using sklearn Library

In [26]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, Y_train)


MultinomialNB()

In [27]:
Y_pred= clf.predict(X_test)

In [30]:
print("report of model by using sklearn:")
print(classification_report(Y_test, Y_pred, target_names=result_num_to_str_map))

report of model by using sklearn:
                          precision    recall  f1-score   support

             alt.atheism       0.65      0.74      0.69        87
           comp.graphics       0.73      0.67      0.70       111
 comp.os.ms-windows.misc       0.79      0.80      0.79       105
comp.sys.ibm.pc.hardware       0.63      0.65      0.64        98
   comp.sys.mac.hardware       0.72      0.71      0.72        89
          comp.windows.x       0.74      0.77      0.76        96
            misc.forsale       0.78      0.73      0.75       115
               rec.autos       0.86      0.87      0.87        86
         rec.motorcycles       0.95      0.91      0.93       102
      rec.sport.baseball       0.95      0.85      0.89       110
        rec.sport.hockey       0.88      0.93      0.90        84
               sci.crypt       0.91      0.90      0.91        93
         sci.electronics       0.81      0.69      0.75        94
                 sci.med       0.92      